# A Multi-Armed Bandit

#### We will now look at a practical example of a Reinforcement Learning problem - the multi-armed bandit problem.

#### The multi-armed bandit is one of the most popular problems in RL:

##### - You are faced repeatedly with a choice among k different options, or actions. After each choice you receive a numerical reward chosen from a stationary probability distribution that depends on the action you selected. Your objective is to maximize the expected total reward over some time period, for example, over 1000 action selections, or time steps.

#### You can think of it in analogy to a slot machine (a one-armed bandit). Each action selection is like a play of one of the slot machine’s levers, and the rewards are the payoffs for hitting the jackpot.

#### Solving this problem means that we can come come up with an optimal policy: a strategy that allows us to select the best possible action (the one with the highest expected return) at each time step.

In [1]:
import numpy as np
import math

#### Primeiro a classe do ambiente é criada, onde as distribuições das recompensas são geradas
#### O ambiente criado permite que os valores das medias sejam aterados, tornando o ambiente um ambiente não estacionário

In [40]:
#Classe com o ambiende 
class MyEnv:
    def __init__(self, arms_num, d_values = [], d_stdev = 1):
        #Numero de braços (ações) do hambiente
        self.arms_num = arms_num 
        #Se não for passado um vetor com as medias das distribuições de recompensa entao valores entre -2 e 2 são criados
        if not d_values:
            self.d_values = [np.random.uniform(-2, 3) for _ in range(self.arms_num)]
        else:
            self.d_values = d_values
        #Desvio padrao das distribuições de recompensas   
        self.d_stdev = d_stdev 
    #---------------------------------------------------------------------------------------------------------
    def getReward(self, action):
        #Dada uma ação retorna um valor aleatorio da distribuição normal de recompensas para aquela ação
        reward = np.random.normal(self.d_values[action], self.d_stdev)
        return reward
    #---------------------------------------------------------------------------------------------------------
    def changeEnv(self, d_change_val = []):
        #Modifica a media das distribuiçõe
        if not d_change_val:
            self.d_values = [np.random.uniform(-2, 2) for _ in range(self.arms_num)] 
        else:
            self.d_values = d_change_val 
    #---------------------------------------------------------------------------------------------------------        
    def printVal(self):
        for i in range(self.arms_num):
            print("Q*(%d)"%i, " = %.5f"%self.d_values[i])
        print("\n")
    #---------------------------------------------------------------------------------------------------------
    def getVal(self):
        return self.d_values

In [51]:
k = 5
#Cria um ambiente com k elementos e valores aleatorios
env = MyEnv(arms_num = k) 
env.printVal()
#Muda os valores para novos valores aleatorios
env.changeEnv()
env.printVal()
#Muda os valores para valores passados
env.changeEnv([3.5, 12.6, 4.6, -99, 1.22])
env.printVal()
#Inverte os valores atuais e os passa como novos valores
env.changeEnv([i*(-1) for i in env.getVal()])
env.printVal()

Q*(0)  = 2.10030
Q*(1)  = -1.89725
Q*(2)  = 2.28026
Q*(3)  = -0.38324
Q*(4)  = 1.61775


Q*(0)  = 0.83117
Q*(1)  = -0.31221
Q*(2)  = -0.51661
Q*(3)  = 1.00935
Q*(4)  = -0.49362


Q*(0)  = 3.50000
Q*(1)  = 12.60000
Q*(2)  = 4.60000
Q*(3)  = -99.00000
Q*(4)  = 1.22000


Q*(0)  = -3.50000
Q*(1)  = -12.60000
Q*(2)  = -4.60000
Q*(3)  = 99.00000
Q*(4)  = -1.22000




#### Calsse do agente

In [11]:
class MyAgent:
    def __init__(self, act_num, step_size = -1, sel_type = 'greedy', eps = 0.1, init_val = 0, ucb_c = 0.1):
        self.act_num = act_num
        self.step_size = step_size
        self.sel_type = sel_type
        self.eps = eps
        self.init_val = init_val
        self.ucb_c = ucb_c
        self.q_values = [init_val for _ in range(act_num)]
        self.n_values = [0 for _ in range(act_num)]
        self.time_steps = 1
        self.total_reward = 0
    #---------------------------------------------------------------------------------------------------------
    def argmax(self, q_values):
        '''Funcao que retorna o indice do valor maximo de um vetor q_values, 
        caso haja mais de um valor maximo entao retorna um deles aletoriamente'''
        top_value = float("-inf") #maior Q(a)
        ties = [] #Acoes que empataram entre o maior valor de Q(a)
        for i in range(self.act_num):
            if(q_values[i] > top_value):     #Se o Q(a) for maior que o maximo ate o momento:
                top_value = q_values[i]          #Substitui o Q(a) maximo pelo valor do atual
                ties = []                        #Zera o vetor de empates das acoes com o Q(a) maximo
                ties.append(i)                   #Adiciona a acao atual no vetor de empates
            elif(q_values[i] == top_value):  #Se o Q(a) for igual ao maximo ate o momento:
                ties.append(i)                   #Adiciona a ação ao vetor de empates
        return np.random.choice(ties)            #Retorna um valor aleatorio do vetor de empates do Q maximo
    #---------------------------------------------------------------------------------------------------------
    def getAction(self):
        
        if(self.sel_type == 'eps-greedy'):
            '''Caso jeja epsilon-greedy gera um valor aleatorio entre 0 e 1 e 
            se for menor que o epsilon entao pega uma ação aleatoria, caso 
            contrario se comporta como o greedy'''
            if(np.random.rand() <= self.eps):
                act = np.random.randint(self.act_num)
            else:
                act = self.argmax(self.q_values)
        
        elif(self.sel_type == 'ucb'):
            '''Se for Upper Confidence Bound utiliza a formula para calcular qual acao 
            é mais indicada baseando na borda superior de incerteza dos Q(a)'''
            ucb_q = [0 for _ in range(self.act_num)]
            for i in range(self.act_num):
                ucb_q[i] = self.q_values[i] + self.ucb_c * math.sqrt(math.log(self.time_steps)/(self.n_values[i]+1))
            act = self.argmax(ucb_q)
        
        else:
            '''Caso seja greedy entao ve qual o maior Q(a) e retorna aquela acao, se tiver 
            mais de um maior entao seleciona aleatriamente entre
            as ações com os Q(a) iguais'''
            act = self.argmax(self.q_values)
        
        self.time_steps += 1
        self.n_values[act] += 1
        self.last_action = act
        return act
    
    #---------------------------------------------------------------------------------------------------------
    def updateValues(self, reward):
        self.total_reward += reward
        '''Atualiza os valore dos Q(a), se o step size for menor que zero usa a 
        media aritimetica dos rewards recebidos, se for maior ou igual a zero usa 
        o weighted average com o step-size passado'''
        if(self.step_size < 0):
            self.q_values[self.last_action] += ((1/self.n_values[self.last_action]) * (reward - self.q_values[self.last_action]))
        else:
            self.q_values[self.last_action] += ((self.step_size) * (reward - self.q_values[self.last_action]))
            
    #---------------------------------------------------------------------------------------------------------
    def printVal(self):
        for i in range(self.act_num):
            print("Q(%d)"%i, " = %.5f"%self.q_values[i])
        print(self.n_values)
        print("%.5f"%self.total_reward)
        print("\n")

#### Primeiro vamos testar diferentes formas de escolher a ação mais vantajosa em um hambiente estacionario
#### A maneira de atualizar os Q(a) usado nessa parte é a media aritimetica dos rewards

In [53]:
k = 5
ep_num = 5000
#CRIAÇÃO DO AMBIENTE
env = MyEnv(arms_num = k, d_stdev=0.5)
print("Actual Values")
env.printVal()
#CRIAÇÃO DOS AGENTES
g_agent = MyAgent(act_num=k)
eg_agent = MyAgent(act_num=k, sel_type="eps-greedy", eps=0.07)
ucb_agent = MyAgent(act_num=k, sel_type="ucb", ucb_c = 3.5)
#TREINAMENTO DOS AGENTES
for i in range(ep_num):
    g_agent.updateValues(env.getReward(g_agent.getAction()))
    eg_agent.updateValues(env.getReward(eg_agent.getAction()))
    ucb_agent.updateValues(env.getReward(ucb_agent.getAction()))
#IMPRESSÃO DOS RESULTADOS   
print("Greedy Agent")
g_agent.printVal()
print("Epsilon Greedy agent")
eg_agent.printVal()
print("UCB Agent")
ucb_agent.printVal()

Actual Values
Q*(0)  = 1.99790
Q*(1)  = -1.05022
Q*(2)  = -0.16734
Q*(3)  = -0.25097
Q*(4)  = 1.16774


Greedy Agent
Q(0)  = 0.00000
Q(1)  = -1.09516
Q(2)  = 0.00000
Q(3)  = -0.03776
Q(4)  = 1.16196
[0, 1, 0, 5, 4994]
5801.54213


Epsilon Greedy agent
Q(0)  = 2.00280
Q(1)  = -0.99276
Q(2)  = -0.17433
Q(3)  = -0.30283
Q(4)  = 1.39257
[4729, 66, 61, 75, 69]
9468.46923


UCB Agent
Q(0)  = 2.01375
Q(1)  = -1.09336
Q(2)  = -0.12597
Q(3)  = -0.13149
Q(4)  = 1.19498
[4840, 9, 20, 19, 112]
9865.55047




#### Agora usando apenas o agente UCB iremos testar deiferentes métoods de atualização dos action values, o primeiro metodo é a media aritimetica dos pesos e o segundo o weighted mean, para isso, no meio do treinamento os valores do ambiente irão mudar para novos valores aleatorios

In [55]:
k = 5
ep_num = 5000
#CRIAÇÃO DO AMBIENTE
env = MyEnv(arms_num = k, d_stdev=0.5)
print("Initial Values")
env.printVal()
#CRIAÇÃO DOS AGENTES
s_agent = MyAgent(act_num=k, sel_type="ucb", ucb_c = 3.5)
ns_agent = MyAgent(act_num=k, sel_type="ucb", ucb_c = 3.5, step_size=0.3)
#TREINAMENTO DOS AGENTES
for i in range(ep_num):
    s_agent.updateValues(env.getReward(s_agent.getAction()))
    ns_agent.updateValues(env.getReward(ns_agent.getAction()))
    if(i == ep_num/2):
        #env.changeEnv()
        env.changeEnv([i*(-1) for i in env.getVal()])
#IMPRESSÃO DOS RESULTADOS   
print("Mean Agent")
s_agent.printVal()
print("Weighted Mean agent")
ns_agent.printVal()
print("Changed Values")
env.printVal()

Initial Values
Q*(0)  = 2.51600
Q*(1)  = -1.28748
Q*(2)  = 0.59557
Q*(3)  = -0.87445
Q*(4)  = 1.85780


Mean Agent
Q(0)  = 1.33225
Q(1)  = 1.25677
Q(2)  = 0.09703
Q(3)  = 0.75818
Q(4)  = 0.75611
[3057, 1527, 51, 181, 184]
6273.09736


Weighted Mean agent
Q(0)  = -0.36970
Q(1)  = 1.29762
Q(2)  = -0.61435
Q(3)  = 0.16120
Q(4)  = -0.13561
[2095, 2242, 49, 268, 346]
8905.13537


Changed Values
Q*(0)  = -2.51600
Q*(1)  = 1.28748
Q*(2)  = -0.59557
Q*(3)  = 0.87445
Q*(4)  = -1.85780




16